In [9]:
import pandas as pd
df = pd.read_csv('C:/Users/dstoc/Documents/Python Scripts/Fixed Income Dashboards/reduced_data_cleaned.csv')

In [20]:
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from math import sqrt


df2 = df.copy()

# Assuming 'Date' is the name of the column containing date information
df2['Date'] = pd.to_datetime(df2['Date'])
df2.set_index('Date', inplace=True)

# Define the features (independent variables) and target (dependent variable)
features = list(df2)

# Define the lag order (number of lags to create)
lag_order = 6  # Example: Creating lagged values for the past n periods

for feature in features:
    for lag in range(1, lag_order):
        df2[f'{feature}_lag{lag}'] = df2[feature].shift(lag)

# Drop rows with missing values in the lagged variables
df2 = df2.dropna()
# Define the number of periods to lead the target variable (1 period in this case)
lead_periods = 1

# Create a new variable representing the 'lead' of the target
df2['YIELD_weekly_percent_change_10 YR'] = df2['YIELD_weekly_percent_change_10 YR'].shift(-lead_periods)

# Drop rows with missing values in the features or target
df2 = df2.dropna(subset=features + ['YIELD_weekly_percent_change_10 YR'])

# Split the data into features and the new target variable 'YIELD_weekly_percent_change_10 YR'
X = df2[features]
y = df2['YIELD_weekly_percent_change_10 YR']

C:\Users\dstoc\AppData\Local\Temp\ipykernel_18160\14027016.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[f'{feature}_lag{lag}'] = df2[feature].shift(lag)
C:\Users\dstoc\AppData\Local\Temp\ipykernel_18160\14027016.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df2[f'{feature}_lag{lag}'] = df2[feature].shift(lag)
C:\Users\dstoc\AppData\Local\Temp\ipykernel_18160\14027016.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor perfo

In [22]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
import numpy as np

# Define the features (independent variables) and target (dependent variable)
features = list(df2.select_dtypes(include=['float64', 'int64']).columns)

# Rest of the code remains the same
X = df2[features].replace([np.inf, -np.inf], np.nan).dropna()
y = df2['YIELD_weekly_percent_change_10 YR']

# Scale features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize the Gradient Boosting Regressor
gbm = GradientBoostingRegressor(n_estimators=100, random_state=248, learning_rate=0.01, subsample=0.75, 
                                max_depth=3, min_samples_leaf=2)

# Define the initial training size (e.g., 70% of the data)
initial_train_size = int(0.7 * len(df2))

# Create lists to store the predictions for each fold
y_pred_train = []
y_pred_test = []

# Create a function for expanding rolling window one-step-ahead cross-validation
def expanding_rolling_cv(X, y, model, initial_train_size):
    rmse_scores = []

    for i in range(initial_train_size, len(X)):
        X_train = X.iloc[:i]
        y_train = y.iloc[:i]

        # Train the model on the training data
        model.fit(X_train, y_train)

        # Append the prediction to the appropriate list
        if i == initial_train_size:
             # Record predictions from the training period
            y_pred = model.predict(X.iloc[:i])
            y_pred_train.extend(y_pred[:])
        else:
             # Make one-step-ahead prediction for the next period
            y_pred = model.predict(X.iloc[i:i+1])
            y_pred_test.append(y_pred[0])

            # Extract the actual target value for the next period
            y_actual = y.iloc[i]

            # Calculate RMSE for this fold
            rmse = sqrt(mean_squared_error([y_actual], [y_pred]))
            rmse_scores.append(rmse)

    return rmse_scores

# Perform expanding rolling window cross-validation and get RMSE scores
rmse_scores = expanding_rolling_cv(X, y, gbm, initial_train_size)

# Print the RMSE scores for each fold
for i, rmse in enumerate(rmse_scores, start=1):
    print(f'Fold {i} RMSE: {rmse:.4f}')

# Calculate the mean RMSE across all folds
mean_rmse = np.mean(rmse_scores)
print(f'Mean RMSE: {mean_rmse:.4f}')

# Use the predictions made during cross-validation as predictions of y
y_pred_cv = y_pred_train + y_pred_test

# Split the predictions into training and testing periods (for plotting)
y_train_pred = y_pred_cv[:initial_train_size]
y_test_pred = y_pred_cv[initial_train_size:]


ValueError: Input y contains infinity or a value too large for dtype('float64').